In [1]:
import numpy as np
import colour
import scipy.io
import cv2 as cv
import warnings

np.set_printoptions(precision=1)

# 1. Сбор данных

Из предложенного <a href="http://cvil.eecs.yorku.ca/projects/public_html/illuminant/illuminant.html" target="_blank">датасета</a> я буду использовать изображения двух камер: Canon EOS-1Ds Mark III и Canon EOS 600D. (первую камеру я для краткости буду называть Mark, а вторую --- Canon). Будем преобразовывать изображения камеры Mark в изображения камеры Canon.

Имена всех фотографий в папке train_images. Данные с этих фотографий будут использованы для поиска матрицы преобразования $M$.

In [2]:
canon_names = ['Canon600D_0002', 'Canon600D_0015', 'Canon600D_0016', 'Canon600D_0017', 'Canon600D_0036', 'Canon600D_0037', 'Canon600D_0041', 'Canon600D_0059', 'Canon600D_0062', 'Canon600D_0070', 'Canon600D_0073', 'Canon600D_0097']
mark_names = ['Canon1DsMkIII_0237', 'Canon1DsMkIII_0250', 'Canon1DsMkIII_0251', 'Canon1DsMkIII_0252', 'Canon1DsMkIII_0120', 'Canon1DsMkIII_0121', 'Canon1DsMkIII_0125', 'Canon1DsMkIII_0143', 'Canon1DsMkIII_0146', 'Canon1DsMkIII_0154', 'Canon1DsMkIII_0157', 'Canon1DsMkIII_0180']

In [3]:
len(canon_names)

12

Изображения хранятся в координатах sRGB. Функция `decode_image` инвертирует это нелинейное преобразование. Я не стал сам писать формулу преобразования в sRGB и обратную к ней. Вместо этого я воспользовался <a href="https://colour.readthedocs.io/en/develop/generated/colour.models.eotf_sRGB.html" target="_blank">функцией</a> из библиотеки colour.

In [4]:
def decode_image(im):
              
    Float_img = im.astype(np.float32)                 #  convert image to float
    
    Normalised_img = Float_img/255.0              #  scale image to [0, 1]

    Decoded_img=colour.models.eotf_sRGB(Normalised_img)       #  inverse  gamma  correction
    
    return Decoded_img               # float  image  in  linear space

Функция `load_image_data` принимает на вход одно jpg изображение и возвращает в переменной `colours` цвета (в линейных координатах) всех 24 квадратов цветовой мишени.

В <a href="http://cvil.eecs.yorku.ca/projects/public_html/illuminant/illuminant.html" target="_blank">датасете</a> для каждого изображения есть файлы mask.txt, color.txt, в которых записаны координаты всех 24 квадратов и их цвета. 
Проблема состоит в том, что эти файлы соответствуют изображениям из датасета в формате png, а не jpg. 

В формате png в датасете хранятся почти 'сырые' изображения, в которых не установлен баланс белого. При визуализации таких изображений получается, либо черное, либо темно зеленое изображение. 

Таким образом, мы не можем использовать файл color.txt. Вместо этого воспользуемся файлом mask.txt, вычислим координаты всех 24 квадратов и вручную посчитаем их средние цвета. 

При этом, как сказано в <a href="http://cvil.eecs.yorku.ca/projects/public_html/illuminant/readme.txt" target="_blank">описании датасета</a>, размер jpg изображений вдвое больше, чем png. Это мы тоже будем учитывать.

In [5]:
def load_image_data(name, visualize = False):
    
    if name.startswith('Canon600D_'):
        camera = 'Canon600D'
    elif name.startswith('PanasonicGX1_'):     
        camera = 'PanasonicGX1'
    elif name.startswith('SonyA57_'):     
        camera = 'SonyA57'
    elif name.startswith('Canon1DsMkIII_'):     
        camera = 'Canon1DsMkIII'
    else: 
        raise ValueError('invalid name')
        
        
    img = cv.imread('./train_images/' + name + '.jpg', cv.IMREAD_UNCHANGED)            # read image
    Decoded_img = decode_image(img)
    
    num = int(name[-4:])         #  number of image                
    
    mat = scipy.io.loadmat('./train_images/data/' + camera + '_gt.mat')
    list(mat['CC_coords'][num-1])              # numbers start with 1
    
    y_l, y_r, x_l, x_r = list(2*mat['CC_coords'][num-1])                          # CC_coords  for  png  images
    
    roi = [y_l, y_r, x_l, x_r]

    if visualize == True:
        cv.imshow("Color checker", img[y_l:y_r, x_l:x_r, :])                             # сначала y, потом x       
        cv.waitKey(0)     
    
    with open('./train_images/data/' + name + '_mask.txt') as f:
        lines = f.readlines()
    
    x = []
    y = []
    for i in range(1, len(lines)): 
        L = lines[i][:-1].split(',')
        if i % 2 == 1: 
            x.append([float(elem) for elem in L])
        if i %2 == 0:
            y.append([float(elem) for elem in L])
            
    colours = []
    for i in range(24):
        b = int(max(x[i]))  
        a = int(min(x[i]))
        B = int(max(y[i]))  
        A = int(min(y[i]))

        l = b - a
        h = B - A

        if h < 25 or l < 25:
            raise ValueError('image ' + name + ' have too small color patches. Cannot be processed')
            
        im = img[y_l + 2*A+10:y_l + 2*B-10, x_l + 2*a+10:x_l + 2*b-10, :]                        
        
        std = np.std(im, axis=(0, 1)) 
        
        if np.max(std) > 5.0:
            warnings.warn('image ' + name + ', patch ' + str(i) + ' wrongly detected')
            
    
        im = Decoded_img[y_l + 2*A+10:y_l + 2*B-10, x_l + 2*a+10:x_l + 2*b-10, :]                           
    
        mean = np.mean(im, axis=(0, 1))                                                             # BGR
        
        colours.append(mean)
        
    return np.array(colours), x, y, roi

Функция `check` используется только для проверки, что все работает правильно

In [6]:
def check(name, i, x, y, roi):
    #i = 9
    img = cv.imread('./train_images/' + name + '.jpg', cv.IMREAD_UNCHANGED)
    y_l, y_r, x_l, x_r = roi
    
    # cv.imshow("image", img[y_l:y_r, x_l:x_r, :])                                     
    # cv.waitKey(0)     
      
    b = int(max(x[i]))  
    a = int(min(x[i]))
    B = int(max(y[i]))  
    A = int(min(y[i]))

    l = b - a
    h = B - A

    if h < 25 or l < 25:
        raise ValueError('image ' + name + ' have too small color patches')
        
    
    im = img[y_l + 2*A+10:y_l + 2*B-10, x_l + 2*a+10:x_l + 2*b-10, :]
    
    cv.imshow("Color patch", im)                          
    cv.waitKey(0)  
    
    mean = np.mean(im, axis=(0, 1))                             # BGR

    std = np.std(im, axis=(0, 1)) 

    if np.max(std) > 5.0:
        warnings.warn('image ' + name + ', patch ' + str(i) + ' wrongly detected')
    
    print('patch mean: ', mean)
    print('patch std: ', std)
    print('patch size: ', h, l)
    
    return

Функция `data_matrix` нужна для того чтобы получить матрицы $R$, $Q$ (я использую обозначения из <a href="https://core.ac.uk/download/pdf/41988521.pdf" target="_blank">статьи</a> Финлейсона). По этим матрицам затем будет вычисляться матрица преобразования $M$ с помощью метода наименьших квадратов.

In [7]:
def data_matrix(names):
    
    data = []
    for name in names:
        
        colours, x, y, roi = load_image_data(name)
        data.append(colours)
        
    return np.vstack(data)

In [8]:
# experiments

colours, x, y, roi = load_image_data(canon_names[-1], visualize = True)

In [9]:
check(canon_names[-1], 23, x, y, roi)

patch mean:  [ 185.6  192.8  111.1]
patch std:  [ 1.3  1.5  2.4]
patch size:  42 48


In [10]:
colours.shape

(24, 3)

# 2. Алгоритм цветовой коррекции

Функция `expansion` принимает на вход массив размера $[..., \, 3]$ и возвращает массив размера $[..., \, D]$, где $D$ зависит от аргумента `degree`. По каждому вектору $(r, \, g, \, b)$ находится вектор размерности $D$, состоящий из координат Финлейсона или полиномиальных координат.

In [11]:
def expansion(RGB, degree, method):

    if method == 'Finlayson':
        b = True
    elif method == 'Polynomial':
        b = False
    else:
        raise ValueError('invalid method')
        
    RGB = RGB.astype(float)
        
    r, g, b = [RGB[..., x] for x in range(RGB.shape[-1])]
    
    if degree not in [1, 2, 3]:
        raise ValueError('invalid degree: only degree <= 3 is available')
    
    if degree == 1:
        return RGB
    
    elif degree == 2:
        if method == 'Finlayson':
            return np.stack([
                r,
                g,
                b,
                np.power(r * g, 1 / 2),
                np.power(g * b, 1 / 2),
                np.power(r * b, 1 / 2),
            ], axis = -1)
        
        if method == 'Polynomial':
            return np.stack([
                r,
                g,
                b,
                r ** 2,
                g ** 2,
                b ** 2,
                r * g,
                g * b,
                r * b,
            ], axis = -1)
    
    elif degree == 3:
        if method == 'Finlayson':
            return np.stack([
                r,
                g,
                b,
                np.power(r * g, 1 / 2),
                np.power(g * b, 1 / 2),
                np.power(r * b, 1 / 2),
                np.power(r * g ** 2, 1 / 3),
                np.power(g * b ** 2, 1 / 3),
                np.power(r * b ** 2, 1 / 3),
                np.power(g * r ** 2, 1 / 3),
                np.power(b * g ** 2, 1 / 3),
                np.power(b * r ** 2, 1 / 3),
                np.power(r * g * b, 1 / 3),
            ], axis = -1)
    
        if method == 'Polynomial':
            return np.stack([
                r,
                g,
                b,
                r ** 2,
                g ** 2,
                b ** 2,
                r * g,
                g * b,
                r * b,
                r ** 3,
                g ** 3,
                b ** 3,
                r * g ** 2,
                g * b ** 2,
                r * b ** 2,
                g * r ** 2,
                b * g ** 2,
                b * r ** 2,
                r * g * b,
            ], axis = -1)

Функция `fit` вычисляет матрицу преобразования $M$ по формуле из статьи Финлейсона. (np.linalg.pinv --- Moore-Penrose inverse)

In [12]:
def fit(R, Q, degree, method):
    
    R_e = expansion(R, degree, method)
        
    return np.dot(np.transpose(Q), np.linalg.pinv(np.transpose(R_e)))

Функция `predict` применяет матрицу $M$ к изображению `im`. 

Сначала изображение `im` переводится из sRGB координат в линейные координаты, а затем обратно.

In [13]:
def predict(im, M, degree, method):
    
    Decoded_img = decode_image(im)
    
    shape = Decoded_img.shape

    RGB = np.reshape(Decoded_img, (-1, 3))

    RGB_e = expansion(RGB, degree, method)
    
    Corrected_img =  np.reshape(np.transpose(np.dot(M, np.transpose(RGB_e))), shape)
    
    Encoded_img = colour.models.eotf_inverse_sRGB(Corrected_img)
    
    Encoded_img = 255.0 * Encoded_img

    return Encoded_img.astype(int)

In [14]:
def fit_predict(im, R, Q, degree, method):
    
    M = fit(R, Q, degree, method)
    
    return predict(im, M, degree, method)

# 3. Применение

Имена всех фотографий из папки train_images

In [15]:
canon_names = ['Canon600D_0002', 'Canon600D_0015', 'Canon600D_0016', 'Canon600D_0017', 'Canon600D_0036', 'Canon600D_0037', 'Canon600D_0041', 'Canon600D_0059', 'Canon600D_0062', 'Canon600D_0070', 'Canon600D_0073', 'Canon600D_0097']
mark_names = ['Canon1DsMkIII_0237', 'Canon1DsMkIII_0250', 'Canon1DsMkIII_0251', 'Canon1DsMkIII_0252', 'Canon1DsMkIII_0120', 'Canon1DsMkIII_0121', 'Canon1DsMkIII_0125', 'Canon1DsMkIII_0143', 'Canon1DsMkIII_0146', 'Canon1DsMkIII_0154', 'Canon1DsMkIII_0157', 'Canon1DsMkIII_0180']

In [16]:
Q = data_matrix(canon_names)
R = data_matrix(mark_names)

In [17]:
R.shape
# R[0]

(288, 3)

Изображения камеры Canon в среднем значительно более яркие, чем изображения камеры Mark. Следовательно, алгоритм цветовой коррекции должен делать изображения камеры Mark более яркими. Проверим это

In [18]:
# experiment

degree = 1
method = 'Finlayson'

predict_mean = []
mark_mean = []
canon_mean = []

for name in mark_names[:3]:
    
    mark_img = cv.imread('./train_images/' + name + '.jpg', cv.IMREAD_UNCHANGED)
    
    predict_img = fit_predict(mark_img, R, Q, degree, method)
    
    predict_mean.append(np.mean(predict_img))
    
    mark_mean.append(np.mean(mark_img))
    
for name in canon_names[:3]:
    
    canon_img = cv.imread('./train_images/' + name + '.jpg', cv.IMREAD_UNCHANGED)
    
    canon_mean.append(np.mean(canon_img))


print('mean brightness of predicted images: ', np.array(predict_mean))
print('mean brightness of mark images: ', np.array(mark_mean))
print('mean brightness of canon images: ', np.array(canon_mean))


mean brightness of predicted images:  [ 182.4  116.2  137.5]
mean brightness of mark images:  [ 159.6  100.4  119.8]
mean brightness of canon images:  [ 177.1  116.3  150.5]


Посмотрим на матрицу цветовой коррекции $M$ 

In [19]:
# experiment

degree = 1
method = 'Finlayson'

M = fit(R, Q, degree, method)
print(M)

[[ 1.6 -0.3  0. ]
 [ 0.   1.3 -0. ]
 [ 0.1 -0.2  1.4]]


### Загрузка результатов работы алгоритмов

Имена фотографий из папки test_images

In [20]:
test_mark_names = ['1_Mark', '2_Mark', '3_Mark', '4_Mark']

Функция `write_result` применяет преобразование к тестовому изображению и сохраняет результат в папке results

In [21]:
def write_result(im, num, R, Q, degree, method):
    
    predict_img = fit_predict(im, R, Q, degree, method)
    
    if degree == 1:
    
        # cv.imwrite('./results/result_' + str(num) + '.jpg', predict_img)
        cv.imwrite('./results/' + str(num) + '_result.jpg', predict_img)
        
    else:
        
        # cv.imwrite('./results/result_' + str(num) + ' (degree=' + str(degree) + ', ' + method + ').jpg', predict_img)
        cv.imwrite('./results/' + str(num) + '_result (degree=' + str(degree) + ', ' + method + ').jpg', predict_img)
    
    return 

In [22]:
# generate   submission

for name in test_mark_names:
    
    num = int(name[:1])                           #  предполагаем, что   тестовых   изображений   меньше     10
    img = cv.imread('./test_images/' + name + '.jpg', cv.IMREAD_UNCHANGED)
    
    for degree in [1, 2, 3]:
        for method in ['Finlayson', 'Polynomial']:
            if degree == 1 and method == 'Polynomial':
                continue
            
            write_result(img, num, R, Q, degree, method)
            